In [49]:
import pandas as pd
import requests
from json.decoder import JSONDecodeError

In [226]:
# Replace YOUR_API_KEY with your actual API key
api_key = "a06fc0a0-aa21-40a5-8abf-476308868dc9"

# URL of the API endpoint
url = f"https://api.balldontlie.io/v1/players?per_page=100&cursor=3000"


# Headers containing the authorization token
headers = {
    "Authorization": f"{api_key}"
}

# Make the GET request to the API
response = requests.get(url, headers=headers)

In [227]:
response.text

'{"data":[{"id":3001,"first_name":"Steve","last_name":"Alford","position":"","height":"6-2","weight":"183","jersey_number":"4","college":"Indiana","country":"USA","draft_year":1987,"draft_round":2,"draft_number":26,"team":{"id":7,"conference":"West","division":"Southwest","city":"Dallas","name":"Mavericks","full_name":"Dallas Mavericks","abbreviation":"DAL"}},{"id":3002,"first_name":"Greg","last_name":"Butler","position":"","height":"6-11","weight":"240","jersey_number":"45","college":"Stanford","country":"USA","draft_year":1988,"draft_round":2,"draft_number":37,"team":{"id":20,"conference":"East","division":"Atlantic","city":"New York","name":"Knicks","full_name":"New York Knicks","abbreviation":"NYK"}},{"id":3003,"first_name":"Steve","last_name":"Scheffler","position":"","height":"6-9","weight":"250","jersey_number":"55","college":"Purdue","country":"USA","draft_year":1990,"draft_round":2,"draft_number":39,"team":{"id":21,"conference":"West","division":"Northwest","city":"Oklahoma Ci

In [234]:
response.json()['meta']['next_cursor']

50609

In [137]:
(pd.DataFrame(response.json()['data']))[["id", "first_name", "last_name", "position", "height", "weight"]]

,id,first_name,last_name,position,height,weight
0,1,Alex,Abrines,G,6-6,190
1,2,Jaylen,Adams,G,6-0,225
2,3,Steven,Adams,C,6-11,265
3,4,Bam,Adebayo,C-F,6-9,255
4,5,DeVaughn,Akoon-Purcell,G-F,6-5,201
...,...,...,...,...,...,...
95,96,Joe,Chealey,G,6-3,190
96,97,Marquese,Chriss,F,6-9,240
97,98,Gary,Clark,F,6-6,225
98,99,Ian,Clark,G,6-3,175


In [236]:
# Creating a DataFrame
df = pd.DataFrame()

In [238]:
if response.status_code == 200:
    i = 1
    while True:
        print(i)
        # Replace YOUR_API_KEY with your actual API key
        api_key = "a06fc0a0-aa21-40a5-8abf-476308868dc9"
        # Headers containing the authorization token
        headers = {
            "Authorization": f"{api_key}"
        }
        url = "https://api.balldontlie.io/v1/players?per_page=100&cursor={}".format(i)
        response = requests.get(url, headers=headers)
        temp_df_1 = (pd.DataFrame(response.json()['data']))[["id", "first_name", "last_name", "position", "height", "weight"]]
        temp_df_2 = pd.DataFrame(list(pd.DataFrame(response.json()['data'])['team']))
        temp_df_2.rename(columns ={'id':'team_id', 'abbreviation':'team_abbreviation', 'city':'team_city','conference':'team_conference','division':'team_division','full_name':'team_full_name','name':'team_name'}, inplace = True)
        temp_df_3 = pd.concat([temp_df_1, temp_df_2], axis = 1)
        df = pd.concat([df, temp_df_3], ignore_index=True)
        i = response.json()['meta']['next_cursor']
else:
    print('Error', response.status_code)

1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
50661
666762
3547262
4196902
17554000
17895913
17896101
38017713
46389631
46390225
46390819
46391413
46393489
46395565
46397641
46400211
46403275
46404363
46405451
46408515
46411086
56677874
1028203085
1057279425


KeyError: 'next_cursor'

In [241]:
df

,id,first_name,last_name,position,height,weight,team_id,team_conference,team_division,team_city,team_name,team_full_name,team_abbreviation
0,2,Jaylen,Adams,G,6-0,225,1,East,Southeast,Atlanta,Hawks,Atlanta Hawks,ATL
1,3,Steven,Adams,C,6-11,265,11,West,Southwest,Houston,Rockets,Houston Rockets,HOU
2,4,Bam,Adebayo,C-F,6-9,255,16,East,Southeast,Miami,Heat,Miami Heat,MIA
3,5,DeVaughn,Akoon-Purcell,G-F,6-5,201,8,West,Northwest,Denver,Nuggets,Denver Nuggets,DEN
4,6,LaMarcus,Aldridge,F,6-11,250,3,East,Atlantic,Brooklyn,Nets,Brooklyn Nets,BKN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,1057396260,Javon,Small,G,6-3,190,15,West,Southwest,Memphis,Grizzlies,Memphis Grizzlies,MEM
5429,1057396603,John,Tonje,G,6-5,218,29,West,Northwest,Utah,Jazz,Utah Jazz,UTA
5430,1057396605,Alex,Toohey,F,6-8,223,24,West,Pacific,Phoenix,Suns,Phoenix Suns,PHX
5431,1057396966,Amari,Williams,F-C,6-11,250,2,East,Atlantic,Boston,Celtics,Boston Celtics,BOS


In [243]:
df[["height_feet", "height_inches"]] = df["height"].str.split("-", expand=True)

# convert to numeric if you want ints
df["height_feet"] = pd.to_numeric(df["height_feet"], errors="coerce")
df["height_inches"] = pd.to_numeric(df["height_inches"], errors="coerce")

# rename weight → weight_pounds
df = df.rename(columns={"weight": "weight_pounds"})

In [245]:
del df['height']

In [247]:
df

,id,first_name,last_name,position,weight_pounds,team_id,team_conference,team_division,team_city,team_name,team_full_name,team_abbreviation,height_feet,height_inches
0,2,Jaylen,Adams,G,225,1,East,Southeast,Atlanta,Hawks,Atlanta Hawks,ATL,6.0,0.0
1,3,Steven,Adams,C,265,11,West,Southwest,Houston,Rockets,Houston Rockets,HOU,6.0,11.0
2,4,Bam,Adebayo,C-F,255,16,East,Southeast,Miami,Heat,Miami Heat,MIA,6.0,9.0
3,5,DeVaughn,Akoon-Purcell,G-F,201,8,West,Northwest,Denver,Nuggets,Denver Nuggets,DEN,6.0,5.0
4,6,LaMarcus,Aldridge,F,250,3,East,Atlantic,Brooklyn,Nets,Brooklyn Nets,BKN,6.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,1057396260,Javon,Small,G,190,15,West,Southwest,Memphis,Grizzlies,Memphis Grizzlies,MEM,6.0,3.0
5429,1057396603,John,Tonje,G,218,29,West,Northwest,Utah,Jazz,Utah Jazz,UTA,6.0,5.0
5430,1057396605,Alex,Toohey,F,223,24,West,Pacific,Phoenix,Suns,Phoenix Suns,PHX,6.0,8.0
5431,1057396966,Amari,Williams,F-C,250,2,East,Atlantic,Boston,Celtics,Boston Celtics,BOS,6.0,11.0


In [249]:
df.to_csv("Player_info.csv")

In [15]:
df

,id,first_name,last_name,position,height_feet,height_inches,weight_pounds,team_id,team_abbreviation,team_city,team_conference,team_division,team_full_name,team_name
0,149,Keenan,Evans,G,NaN,NaN,NaN,9,DET,Detroit,East,Central,Detroit Pistons,Pistons
1,382,Billy,Preston,F,NaN,NaN,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
2,448,Gary,Trent Jr.,G,NaN,NaN,NaN,25,POR,Portland,West,Northwest,Portland Trail Blazers,Trail Blazers
3,46391370,Jake,Carter,,NaN,NaN,NaN,50,DN,,,,Denver Nuggets,Denver Nuggets
4,2144,Rakeem,Christmas,,NaN,NaN,NaN,12,IND,Indiana,East,Central,Indiana Pacers,Pacers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5224,492,Ante,Zizic,C,6.0,11.0,254.0,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
5225,2639,Jim,Zoet,,NaN,NaN,NaN,9,DET,Detroit,East,Central,Detroit Pistons,Pistons
5226,46405957,Bill,Zopf,,NaN,NaN,NaN,17,MIL,Milwaukee,East,Central,Milwaukee Bucks,Bucks
5227,493,Ivica,Zubac,C,7.0,1.0,240.0,13,LAC,LA,West,Pacific,LA Clippers,Clippers
